### A pyiron workflow to calculate a grain boundary energy using multiple potentials, and compare to literature DFT data.

This notebook __doesn't__ work out of the box if you haven't configured LAMMPS for pyiron

conda install -c conda-forge lammps 

OR

mamba install -c conda-forge lammps

In the python environment in which you are running pyiron

WARNING: In practice/production you should configure a compiled LAMMPS executable that is optimised for your cluster!

In [1]:
from pyiron.project import Project

import numpy as np
import pandas as pd

from ase.lattice.cubic import BodyCenteredCubic as bcc
from ase.build import stack

from pyiron.atomistics.structure.atoms import ase_to_pyiron

In [2]:
pr = Project("GB_calcs")

Create the structure that we're going to use:
a $\Sigma 3 [1 \bar{1}0](111)$ Fe GB

In [3]:
surface1 = [1, 1, 1]
surface2 = [1, 1, -1]
rotation_axis = [1, -1, 0]
element = "Fe"
lc = 2.8318488966083
GB_name = "S3-RA110-S1-11"

# The minimum required length of the cell.
req_length = 15

v1 = list(-np.cross(rotation_axis,surface1))
v2 = list(-np.cross(rotation_axis,surface2)) 

length = 0
n = 0
while length < req_length:
    n += 1
    
    slab1 = bcc(symbol=element, latticeconstant=lc,directions=[rotation_axis,v1,surface1], size=[1,1,n])
    slab2 = bcc(symbol=element, latticeconstant=lc,directions=[rotation_axis,v2,surface2], size=[1,1,n])

    gb = stack(slab1, slab2)
    slab = stack(slab1, slab1)
    length = gb.cell[-1,-1]
    
    # Rattle the gb structure using rattle in ASE to perturb symmetry
    gb.rattle()
    slab.rattle()

Create the pyiron job, assign the structure to the job.

In [4]:
job = pr.create_job(job_type=pr.job_type.Lammps, job_name='Fe_S3_GB')

job.structure = ase_to_pyiron(gb)

We need to see the potentials, pyiron has an inbuilt potential database

It automatically finds the applicable potentials given the structure provided.

In [5]:
job.list_potentials()[:10]

['1997--Ackland-G-J--Fe--LAMMPS--ipr1',
 '1998--Meyer-R--Fe--LAMMPS--ipr1',
 '2001--Lee-B-J--Fe--LAMMPS--ipr1',
 '2001--Lee-B-J--Fe-Cr--LAMMPS--ipr1',
 '2003--Mendelev-M-I--Fe-2--LAMMPS--ipr3',
 '2003--Mendelev-M-I--Fe-5--LAMMPS--ipr1',
 '2004--Ackland-G-J--Fe-P--LAMMPS--ipr1',
 '2004--Zhou-X-W--Fe--LAMMPS--ipr2',
 '2005--Lee-B-J--Fe-Cu--LAMMPS--ipr1',
 '2005--Mendelev-M-I--Al-Fe--LAMMPS--ipr1']

In [6]:
potentials_tested = job.list_potentials()
GB_energy_list = []
for i, potential in enumerate(potentials_tested):
    try:
        job_GB_name = f"S3_GB_{i}"
        job_GB = pr.create_job(job_type=pr.job_type.Lammps, job_name=job_GB_name)
        job_GB.structure = ase_to_pyiron(gb)
        job_GB.potential = potential
        job_GB.calc_minimize(pressure=None,
                            max_iter=10000)
        job_GB.run(delete_existing_job=True)
        
        job_SLAB_name = f"S3_SLAB_{i}"
        job_SLAB = pr.create_job(job_type=pr.job_type.Lammps, job_name=job_SLAB_name)
        job_SLAB.structure = ase_to_pyiron(slab)
        job_SLAB.potential = potential
        job_SLAB.calc_minimize(pressure=None,
                            max_iter=10000)
        job_SLAB.run(delete_existing_job=True)
        
        # Formula for GB energy is E_GB_tot - E_SLAB_tot / 2 * (Interface area)
        # The 2 in the denominator is to account for the second interface present in the cell due to the periodic conditions
        
        GB_energy = (job_GB['output/generic/energy_tot'][-1] - job_SLAB['output/generic/energy_tot'][-1]) \
                    / (2 * job_GB.structure.cell.volume / job_GB.structure.cell[-1,-1]) * 16.02
        
        GB_energy_list.append(GB_energy)
    except:
        GB_energy_list.append(np.nan)

2023-08-11 18:23:41,995 - pyiron_log - WARNING - No HDF5 file found - remove database entry and create new job! S3_GB_0


The job S3_GB_0 was saved and received the ID: 753


2023-08-11 18:23:42,505 - pyiron_log - WARNING - No HDF5 file found - remove database entry and create new job! S3_SLAB_0


The job S3_SLAB_0 was saved and received the ID: 754


2023-08-11 18:23:42,972 - pyiron_log - WARNING - No HDF5 file found - remove database entry and create new job! S3_GB_1


...

The job S3_GB_124 was saved and received the ID: 1026


2023-08-11 18:25:24,982 - pyiron_log - WARNING - Job aborted
2023-08-11 18:25:24,983 - pyiron_log - WARNING - LAMMPS (28 Mar 2023 - Update 1)
Reading data file ...
  orthogonal box = (0 0 0) to (4.0048391 6.9365848 19.619625)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  48 atoms
  read_data CPU = 0.000 seconds
ERROR: KIM ModelCreate failed (src/KIM/pair_kim.cpp:828)
Last command: pair_style kim Sim_LAMMPS_TersoffZBL_ByggmastarGranberg_2020_Fe__SM_958863895234_000

2023-08-11 18:25:25,000 - pyiron_log - WARNING - No HDF5 file found - remove database entry and create new job! S3_GB_126


The job S3_GB_125 was saved and received the ID: 1028


2023-08-11 18:25:25,376 - pyiron_log - WARNING - Job aborted
2023-08-11 18:25:25,378 - pyiron_log - WARNING - LAMMPS (28 Mar 2023 - Update 1)
Reading data file ...
  orthogonal box = (0 0 0) to (4.0048391 6.9365848 19.619625)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  48 atoms
  read_data CPU = 0.000 seconds
ERROR: KIM ModelCreate failed (src/KIM/pair_kim.cpp:828)
Last command: pair_style kim Sim_LAMMPS_TersoffZBL_HenrikssonBjorkasNordlund_2013_FeC__SM_473463498269_000



The job S3_GB_126 was saved and received the ID: 1029


Construct a dataframe which contains the information about GB energy, and their errors with respect to a DFT computed value.

The GB energy of 1.58 J/m^2 is from the value computed in this study:

Mai, H.L., Cui, X.Y., Scheiber, D., Romaner, L. and Ringer, S.P., 2022. The segregation of transition metals to iron grain boundaries and their effects on cohesion. Acta materialia, 231, p.117902.

In [7]:
data = {'potential': potentials_tested, 'GB_energy': GB_energy_list}
df = pd.DataFrame(data)

GB_energy_DFT = 1.58
df["err_DFT"] = np.round(GB_energy_DFT - df["GB_energy"], 3)
df["rel_err_DFT"] = np.round((GB_energy_DFT - df["GB_energy"]) / GB_energy_DFT * 100, 0)
df["GB_energy"] = np.round(df.GB_energy.tolist(), 3)
df = df.dropna(subset="GB_energy")
df

,potential,GB_energy,err_DFT,rel_err_DFT
0,1997--Ackland-G-J--Fe--LAMMPS--ipr1,0.980,0.600,38.0
1,1998--Meyer-R--Fe--LAMMPS--ipr1,1.078,0.502,32.0
2,2001--Lee-B-J--Fe--LAMMPS--ipr1,1.349,0.231,15.0
3,2001--Lee-B-J--Fe-Cr--LAMMPS--ipr1,1.342,0.238,15.0
4,2003--Mendelev-M-I--Fe-2--LAMMPS--ipr3,1.386,0.194,12.0
...,...,...,...,...
63,2021--Starikov-S--Fe--LAMMPS--ipr2,-0.604,2.184,138.0
64,2021--Wen-M--Fe-H--LAMMPS--ipr1,1.403,0.177,11.0
65,2022--Mahata-A--Al-Fe--LAMMPS--ipr1,1.196,0.384,24.0
66,2022--Starikov-S--Fe-Cr-H--LAMMPS--ipr1,1.609,-0.029,-2.0


Order the potentials by the minimum error wrt. DFT!

Now we have a good idea of how to search for an empirical potential which best captures the GB energetics of a specific grain boundary.

You can see that there are some potentials that are very, very accurate at predicting this specific GB energy. 

For a real study you should consider not just a single GB, but validate it across multiple GBs and ideally the specific phenomena you want to study.

In [8]:
df.sort_values(by="err_DFT").head(10)

,potential,GB_energy,err_DFT,rel_err_DFT
66,2022--Starikov-S--Fe-Cr-H--LAMMPS--ipr1,1.609,-0.029,-2.0
27,2010--Malerba-L--Fe--LAMMPS--ipr1,1.549,0.031,2.0
55,2019--Byggmastar-J--Fe-O--LAMMPS--ipr1,1.499,0.081,5.0
40,2013--Henriksson-K-O-E--Fe-C--LAMMPS--ipr1,1.499,0.081,5.0
57,2020--Byggmastar-J--Fe--LAMMPS--ipr1,1.466,0.114,7.0
58,2020--Farkas-D--Fe-Ni-Cr-Co-Al--LAMMPS--ipr1,1.425,0.155,10.0
61,2021--Deluigi-O-R--Fe-Ni-Cr-Co-Cu--LAMMPS--ipr1,1.425,0.155,10.0
50,2018--Farkas-D--Fe-Ni-Cr-Co-Cu--LAMMPS--ipr2,1.425,0.155,10.0
6,2004--Ackland-G-J--Fe-P--LAMMPS--ipr1,1.404,0.176,11.0
9,2005--Mendelev-M-I--Al-Fe--LAMMPS--ipr1,1.404,0.176,11.0
